In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import ParameterGrid
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingClassifier, BaggingRegressor
from sklearn.metrics import mean_squared_error, r2_score, log_loss, accuracy_score

import matplotlib.pyplot as plt

In [2]:
# importing the full dataframe from Assignment 5
merged_df = pd.read_csv("merged_final.csv")

# adding stock value as a column
merged_df['value'] = merged_df['PRC'] * merged_df['VOL']

In [3]:
rf_month = pd.read_sas("ff_factors_monthly.sas7bdat")

rf_month['dateff'] = pd.to_datetime(rf_month['dateff'])
#rf_month = rf_month[rf_month['dateff']]
rf_month = rf_month[rf_month['dateff'].dt.year >= 1980]
rf_month = rf_month[['dateff', 'RF']]

def month_format(month):
    if len(month) == 1:
        month = str(0) + month
        return month
    else:
        return month

rf_month['year'] = rf_month['dateff'].apply(lambda x: str(x.year))
rf_month['month'] = rf_month['dateff'].apply(lambda x: str(x.month))
rf_month['month'] = rf_month['month'].apply(month_format)
rf_month['yyyymm'] = rf_month['year'] + rf_month['month']
rf_month['yyyymm'] = rf_month['yyyymm'].apply(lambda x: float(x))

rf_month = rf_month[['RF', 'yyyymm']]

In [4]:
# adding risk free rate to merged_df
merged_df = merged_df.merge(rf_month, on = ['yyyymm'])

# setting index to permno-yyyymm
merged_df['permno-yyyymm'] = merged_df['permno'].apply(lambda x: str(x)) + merged_df['yyyymm'].apply(lambda x: str(x)[:-2])
merged_df['permno-yyyymm'] = merged_df['permno-yyyymm'].apply(lambda x: int(x))
merged_df.set_index(merged_df['permno-yyyymm'], inplace = True)

# dropping the permno-yyyymm column
merged_df.drop(['permno-yyyymm'], axis= 1, inplace= True)
#merged_df.drop(['Unnamed: 0'], axis= 1, inplace= True)

In [5]:
# using a 60/20/20 split
train, validate, test = \
                        np.split(merged_df.sample(frac=1, random_state=42), 
                        [int(.6*len(merged_df)), int(.8*len(merged_df))])

factors = list(train.columns[9:41])

x_train = train[factors]
y_train = train['RET']

x_validate = validate[factors]
y_validate = validate['RET']

x_test = test[factors]
y_test = test['RET']

In [6]:
# validation function
def validate_model(model_type, param_grid, x_train, y_train, x_validate, y_validate):
    # Special case for LinearRegression because it doesn't have hyperparameters to tune
    if model_type == LinearRegression:
        model = LinearRegression()
        model.fit(x_train, y_train)
        pred = model.predict(x_validate)
        r2 = r2_score(y_validate, pred)
        
        return r2
    else: # The other cases
        
        # Establishses the ParameterGrid
        model_param_grid = ParameterGrid(param_grid)
        
        # Initialize values
        best_MAE = 0
        best_r2 = 1
        best_config = None
        # Iterate through the parameter grid, fit models to the hyperparameters
        # and check for MAE and R2 values
        
        # each param_config in that validation function would represent 1 combination of the possible parameters.
        # for example in Lab 6, when I'm validating for the elastic net regression, I have 
        # 2 possible hyperparameters: alpha and l1_ratio. 
        #alpha can take on values 0.0001, 0.0005, etc, and l1_ratio can take on values 0, 1, 0.01. 
        #So each param_config in the for loop in validate_model would go over 1 possible 
        #combination of the hyperparameter and keep the one that gives us the best MAE/R2
        for param_config in model_param_grid:
            curr_config_MAEs = []
            model = model_type(**param_config)
            model.fit(x_train, y_train)
            pred = model.predict(x_validate)
            MAE = mean_squared_error(y_validate,pred)
            r2 = r2_score(y_validate, pred)
            curr_config_MAEs.append(MAE)
            if best_MAE == 0 or (MAE < best_MAE):
                best_MAE = MAE
                best_config = param_config
            if best_r2 == 1 or (r2 > best_r2):
                best_r2 = r2
        return best_config, best_MAE, best_r2

In [7]:
# Predictions
def pred(model_type, x_train, y_train, x_test, y_test):
    # Fit model and predict 
    model = model_type.fit(x_train, y_train)
    pred = model.predict(x_test)
    
    # Format prediction as DataFrame
    pred_df = pd.DataFrame(pred, columns = ['RET_pred'])
    pred_df.set_index(x_test.index, inplace = True)
    
    r2 = r2_score(y_test, pred)
    return pred_df, r2

In [8]:
def year_extract(index):
    return int(str(index)[5:])
# vectorizes the function
vyear_extract = np.vectorize(year_extract)

In [9]:
# Function to build the portfolio and calculate different performance metrics
def portfolio_build(df_pred, model_name, df_realized):
    # Get the year
    df_pred['yyyymm'] = vyear_extract(df_pred.index.values)
    df_pred.sort_values(by = 'yyyymm', inplace = True)
    
    # Initialize list of values
    long_short_lst = []
    value_lst = []
    
    # Iterate over years
    for time in df_pred['yyyymm'].unique():
        # Subset by year
        df_curr = df_pred[df_pred['yyyymm'] == time]
        # Sort the values by returns
        df_curr = df_curr.sort_values(by = ['RET_pred'], ascending = False)
        size = math.floor(df_curr.shape[0]/10)
        # Get the top/bottom performing stocks
        df_top = df_curr.head(size)
        df_bot = df_curr.tail(size)
        # Get the actual returns of the predicted top/bottom performers
        df_top_realized = df_realized.loc[df_top.index]
        df_bot_realized = df_realized.loc[df_bot.index]
        
    
        # Get the mean returns of these top/bottom performers
        mu_top = df_top_realized['RET'].mean()
        mu_bot = df_bot_realized['RET'].mean()
        
     
        # Get the mean returns by shorting the bottom and going long on the top
        long_short = (mu_top - mu_bot)
        
        value = df_top['value'].sum()
        value = value + df_bot['value'].sum()
        
        long_short_lst.append(long_short)
        value_lst.append(value)
    
    # Get the value for the portfolio
    ls_df = pd.DataFrame(long_short_lst, columns = ['ls_ret'], index = df_pred['yyyymm'].unique())
    ls_df['value'] = value_lst
    
    # Calculating cumulative returns
    # First we +1 to the returns 
    # Then we do the cumulative product
    ls_df['cumulative_ret']= ls_df['ls_ret'] + 1
    ls_df['cumulative_ret'] = ls_df['cumulative_ret'].cumprod()
    
    # Get mean/std/sharpe ratio for the portfolio
    ls_df = pd.merge(ls_df, rf_month, left_index = True, right_index = True)
    ls_df['ls_sub_rf'] = ls_df['ls_ret'] - ls_df['RF']
    ls_sub_rf_mean = ls_df['ls_sub_rf'].mean()*12
    ls_sub_rf_std = ls_df['ls_sub_rf'].std()*math.sqrt(12)
    sharpe_ratio = ls_sub_rf_mean/ls_sub_rf_std*math.sqrt(12)
    
    print("The annualized excess return "+ model_name + " long short portfolio is: " + str(ls_sub_rf_mean))
    print("The annualized std dev of returns of " + model_name + " long short portfolio is: " + str(ls_sub_rf_std))
    print("The anualized Sharpe ratio of " + model_name + " long short portfolio is: " + str(sharpe_ratio))
    
    return ls_df, ls_sub_rf_mean, ls_sub_rf_std, sharpe_ratio

In [10]:
# Parameter grids used for validation
RF_para_grid_dict = {'max_depth': np.arange(1, 50, 1),
               'n_estimators': np.arange(1, 101, 1),
               'max_features': np.arange(1, 36, 1),
               'n_jobs':[-1]}

BGDT_para_grid_dict = {'base_estimator':[DecisionTreeRegressor(max_depth=15)], 
               'n_estimators': np.arange(1, 101, 1),
               'max_features': np.arange(1, 36, 1),
               'n_jobs':[-1],
               'max_samples': np.arange(0.1, 1.1, 0.1),
               }
RSDT_para_grid_dict = {'base_estimator':[DecisionTreeRegressor(max_depth=15)], 
               'n_estimators': np.arange(1, 101, 1),
               'max_features': np.arange(1, 36, 1),
               'max_samples': [1.0],
               'n_jobs':[-1]
               }

In [ ]:
# Sample vali code for one model
# Actual validation is ran on a random sample of 50 as training data, and random sample of 20 as validation
# Caveat: Each model takes a long time--more than 6 hours--to run in a laptop PC
# This part of the code is mannually terminated to save time. Results hence are not optimal

# Sample vali code for one model
# Actual validation is ran on a random sample of 50 as training data, and random sample of 20 as validation
RF_train = merged_df.sample(n=50, random_state = 42)
RF_validate = merged_df.sample(n=20, random_state = 100)

RFx_train = train[factors]
RFy_train = train['RET']

RFx_validate = validate[factors]
RFy_validate = validate['RET']

RF_para_grid_dict = {'max_depth': np.arange(1, 50, 1),
               'n_estimators': np.arange(1, 100, 1),
               'max_features': np.arange(1, 32, 1),
               'n_jobs':[-1]}

#RF_para_grid = ParameterGrid(RF_para_grid_dict)
RF_best_config, RF_best_MAE, RF_best_r2 = validate_model(RandomForestRegressor, RF_para_grid_dict, RFx_train, RFy_train, RFx_validate, RFy_validate)
RF_para_grid_dict = {'max_depth': np.arange(1, 50, 1),
               'n_estimators': np.arange(1, 100, 1),
               'max_features': np.arange(1, 32, 1),
               'n_jobs':[-1]}

print('Best Config' + str(RF_best_config))
print('Validation R2: '+ str(RF_best_r2))

In [ ]:
# Parameter values gotten from validation -- from a previous exercise from Nasdaq 100 firms only, instead of the full sample used
# Make predictions 
RF_pred_df, RF_test_r2 = pred(RandomForestRegressor(max_depth= 45, max_features= 7, n_estimators= 1, n_jobs= -1),\
                x_train, y_train, x_test, y_test)
BGDT_pred_df, BGDT_test_r2 = pred(BaggingRegressor(base_estimator = DecisionTreeRegressor(max_depth = 45),\
                              max_features = 9, max_samples= 0.4, n_estimators = 2,\
                              n_jobs = -1), x_train, y_train, x_test, y_test)
RSDT_pred_df, RSDT_test_r2 = pred(BaggingRegressor(base_estimator = DecisionTreeRegressor(max_depth = 45),\
                               max_features = 1, max_samples= 1.0, n_estimators = 19, n_jobs = -1),\
                                 x_train, y_train, x_test, y_test)

In [ ]:
# Setting rf_month index to match ls_df later
rf_month['yyyymm'] = rf_month['yyyymm'].apply(lambda x: int(str(x)[:6]))
rf_month.set_index(rf_month['yyyymm'], inplace= True)

In [ ]:
# adds the stock values to the predicted DataFrames
RF_pred_df = pd.merge(RF_pred_df, merged_df['value'], left_index=True, right_index=True)
BGDT_pred_df = pd.merge(BGDT_pred_df, merged_df['value'], left_index=True, right_index=True)
RSDT_pred_df = pd.merge(RSDT_pred_df, merged_df['value'], left_index=True, right_index=True)

In [ ]:
# ls_df, ls_sub_rf_mean, ls_sub_rf_std, sharpe_ratio
RF_ls_df, RF_ls_sub_rf_mean, RF_ls_sub_rf_std, RF_sharpe_ratio = portfolio_build(RF_pred_df, "RF", merged_df)

In [ ]:
BGDT_ls_df, BGDT_ls_sub_rf_mean, BGDT_ls_sub_rf_std, BGDT_sharpe_ratio = portfolio_build(BGDT_pred_df, "BGDT", merged_df)

In [ ]:
RSDT_ls_df, RSDT_ls_sub_rf_mean, RSDT_ls_sub_rf_std, RSDT_sharpe_ratio = portfolio_build(RSDT_pred_df, "RSDT", merged_df)

In [ ]:
# Get values from previous assignment for comparison
assignment6_metrics = pd.read_csv("assignment6_metrics.csv")
assignment6_values = pd.read_csv("assignment6_values.csv")

assignment6_values.drop('Unnamed: 0', axis= 1, inplace= True)
assignment6_values.set_index(RF_ls_df.index, inplace= True)

In [ ]:
assignment6_values

In [ ]:
plt.plot(RF_ls_df.index, RF_ls_df['cumulative_ret'], label = 'RF')
plt.plot(BGDT_ls_df.index, BGDT_ls_df['cumulative_ret'], label = 'BGDT')
plt.plot(RSDT_ls_df.index, RSDT_ls_df['cumulative_ret'], label = 'RSDT')
plt.plot(RF_ls_df.index, assignment6_values['slr_cum_ret'], label = 'SLR')
plt.plot(RF_ls_df.index, assignment6_values['en_cum_ret'], label = 'EN')
plt.plot(RF_ls_df.index, assignment6_values['pls_cum_ret'], label = 'PLS')

plt.yscale('log')
plt.legend()
plt.title("Rolling average of portfolio values vs Time")
    
plt.show()

In [22]:
# The r2 scores are attained if you ran the validation_model function
# I've just assigned them here since I'm not running the full function which takes time

RF_best_r2 = 0.4023358998970291
BGDT_best_r2 = 0.38203966204801354
RSDT_best_r2 = 0.20295386908083823

In [49]:
from tabulate import tabulate

table = [['Type', 'Avg Ret', 'Std. Dev', 'SR', 'R^2'],\
         [assignment6_metrics['Type'][0], assignment6_metrics['Avg Ret'][0], assignment6_metrics['Std. Dev'][0],\
          assignment6_metrics['SR'][0], assignment6_metrics['R^2'][0]],
        [assignment6_metrics['Type'][1], assignment6_metrics['Avg Ret'][1], assignment6_metrics['Std. Dev'][1],\
          assignment6_metrics['SR'][1], assignment6_metrics['R^2'][1]],
        [assignment6_metrics['Type'][2], assignment6_metrics['Avg Ret'][2], assignment6_metrics['Std. Dev'][2],\
          assignment6_metrics['SR'][2], assignment6_metrics['R^2'][2]],
         ['RF', RF_ls_sub_rf_mean, RF_ls_sub_rf_std, RF_sharpe_ratio, RF_best_r2],
         ['BGDT', BGDT_ls_sub_rf_mean, BGDT_ls_sub_rf_std, BGDT_sharpe_ratio, BGDT_best_r2],
         ['RSDT', RSDT_ls_sub_rf_mean, RSDT_ls_sub_rf_std, RSDT_sharpe_ratio, RSDT_best_r2]]

print(tabulate(table, headers='firstrow'))

Type        Avg Ret    Std. Dev          SR         R^2
------  -----------  ----------  ----------  ----------
SLR      0.271556      0.28493    3.30151    0.00460928
EN       0.266641      0.297459   3.10521    0.00483712
PLS      0.244333      0.290277   2.91582    0.00470622
RF      -0.00854582    0.225442  -0.131313   0.402336
BGDT     0.00382652    0.202823   0.0653549  0.38204
RSDT    -0.0856384     0.216177  -1.3723     0.202954


In [51]:
# Export results for later use

data = table[1:]
results = pd.DataFrame(data, columns= ['Type', 'Avg Ret', 'Std. Dev', 'SR', 'R^2'])

rol_val = pd.concat([assignment6_values['slr_cum_ret'],assignment6_values['en_cum_ret'], assignment6_values['pls_cum_ret'], \
                     RF_ls_df['cumulative_ret'], BGDT_ls_df['cumulative_ret'], RSDT_ls_df['cumulative_ret']],\
                     axis = 1) 
rol_val.columns = ['slr_cum_ret', 'en_cum_ret', 'pls_cum_ret', 'RF_cum_ret', 'BGDT_cum_ret', 'RSDT_cum_ret']

In [54]:
results.to_csv("assignment7_metrics.csv")
rol_val.to_csv("assignment7_values.csv")